# Code Challenge IMA 205 : Cardiac Pathology Prediction 
### Gaël Marcheville, 2nd year student at Télécom Paris 

## 0. Readme

This notebook is meant to be run on Jupyter Notebook. I only tested it on VSCode, but it should work on any other IDE. The code is written in Python 3.11. The notebook should be run at the root of the project, so that the data is in the same folder as the notebook (Train and Test folders). Please contact me if you have any problem running the notebook.

It create a file called `submission.csv` in the same folder as the notebook. This file is the one to be submitted on the challenge website.

## 1. Import

Below are all the libraries used for this project. The main libraries are pandas for data processing, numpy and scipy for calculations and statistics, and scikit-learn for machine learning algorithms.

### a. Import libraries

In [1]:
import cv2 as cv
print("CV version : ",cv.__version__) # 4.7.0 for me
import nibabel as nib
print("NIB version : ",nib.__version__) # 5.1.0 for me
import numpy as np
print("Numpy version : ",np.__version__) # 1.23.5 for me    
import pandas as pd
print("Pandas version : ",pd.__version__) # 2.0.1 for me
from scipy.ndimage import binary_fill_holes
from scipy.stats import kurtosis, skew
print("Scipy version : ",pd.__version__) # 2.0.1 for me
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
print("Scikit-learn version : ",pd.__version__) # 2.0.1 for me

CV version :  4.7.0
NIB version :  5.1.0
Numpy version :  1.23.5
Pandas version :  2.0.1
Scipy version :  2.0.1
Scikit-learn version :  2.0.1


### b. Load data

In [2]:
dataTrain = pd.read_csv('./metaDataTrain.csv', sep = ',')
dataTest = pd.read_csv('./metaDataTest.csv', sep = ',')

Id = dataTest['Id']
class_names = ['0','1','2','3','4'] 

## 2. Algorithms to compute features
### a. Compute Volume

In [3]:
def compute_vol(binary_img, voxel_size = (1,1,1)):
    binary_img = binary_img.astype(np.uint8)
    return np.sum(binary_img) * np.prod(voxel_size)

### b. Segment the left ventricle for test set

In [4]:
def mask_contour(binary_img):
    mask = np.zeros(binary_img.shape)
    for i in range (binary_img.shape[2]):
        mask[:,:,i] = binary_fill_holes(binary_img[:,:,i]).astype(int)
        mask[:,:,i] -= binary_img[:,:,i]
    return mask

### c. Compute the thickness, circularity and circumference of the myocardium


In [5]:
def thickness_myo(binary_img, voxel_size = (1,1,1)):
    binary_img = binary_img.astype(np.uint8)
    thickness = np.zeros(binary_img.shape[2])
    for k in range (binary_img.shape[2]):
        dist_transform = cv.distanceTransform(binary_img[:,:,k], cv.DIST_L2, 5) * (np.sqrt(voxel_size[0] + voxel_size[1]))
        thickness[k] = (dist_transform.max() - dist_transform.min()) 
    return np.mean(thickness), np.std(thickness), np.max(thickness), np.min(thickness)

In [6]:
def circularity(binary_img, voxel_size = (1,1,1)):
    binary_img = binary_img.astype(np.uint8)
    circularity = np.zeros((binary_img.shape[2],))
    for k in range(binary_img.shape[2]):
        contours, hierarchy = cv.findContours(binary_img[:,:,k], cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:
            cnt = contours[0]
            area = cv.contourArea(cnt)
            perimeter = cv.arcLength(cnt,True) * (np.sqrt(voxel_size[0] + voxel_size[1]))
            if perimeter != 0:
                circularity[k] = 4*np.pi*area/(perimeter*perimeter)
            else:
                circularity[k] = 0
    return np.mean(circularity)

In [7]:
def circumference_myo(binary_img, voxel_size = (1,1,1)):
    binary_img = binary_img.astype(np.uint8)
    circumference = np.zeros(binary_img.shape[2])
    for k in range (binary_img.shape[2]):
        contours, _ = cv.findContours(binary_img[:,:,k], cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            circumference[k] = cv.arcLength(cnt, True) * (np.sqrt(voxel_size[0] + voxel_size[1]))
    return np.mean(circumference), np.max(circumference), np.min(circumference)

## 3. Add the features to the dataframe

The following code adds the features to the dataframe. It takes as input the dataframe, and returns the dataframe with the features. It uses the functions defined above. XTrainImg and XTestImg are the images of the training and test set. XTrain and XTest are whole features of the training and test set. 

In [8]:
XTrain = dataTrain.drop(['Category','Id'], axis=1)
XTest = dataTest.drop(['Id'], axis=1)
yTrain = dataTrain['Category']

shape = nib.load('.\\Train\\001\\001_ED.nii').get_fdata().shape

XTrainImg = np.empty((4, 100), dtype=object)
XTestImg = np.empty((4, 100), dtype=object)

#adding data for train
for k in range (0,XTrain.shape[0]):
    
    k_str = '{:03d}'.format(k + 1)
    ED = '.\\Train\\' + k_str + '\\' + k_str + '_ED.nii'
    ED_seg = '.\\Train\\' + k_str + '\\' + k_str + '_ED_seg.nii'
    ES = '.\\Train\\' + k_str + '\\' + k_str + '_ES.nii'
    ES_seg = '.\\Train\\' + k_str + '\\' + k_str + '_ES_seg.nii'

    XTrainImg[0,k] = nib.load(ED)
    XTrainImg[1,k] = nib.load(ED_seg)
    XTrainImg[2,k] = nib.load(ES)
    XTrainImg[3,k] = nib.load(ES_seg)

    # Instant features
    # ED 
    data = XTrainImg[1,k].get_fdata()
    zoom = np.array(XTrainImg[1,k].header.get_zooms())
    XTrain.loc[k,'rv_ED'] = compute_vol((data == 1), zoom) #1 - Left ventricle cavity
    XTrain.loc[k,'myo_ED'] = compute_vol((data == 2), zoom) #2 - Myocardium
    XTrain.loc[k,'lv_ED'] = compute_vol((data == 3), zoom) #3 - Right ventricle cavity
    XTrain.loc[k,'thick_ED_mean'], XTrain.loc[k,'thick_ED_std'], XTrain.loc[k,'thick_ED_max'], XTrain.loc[k,'thick_ED_min'] = thickness_myo((data == 2),zoom) #4 - Myocardium thickness Mean
    XTrain.loc[k,'circul_ED'] = circularity((data == 2),zoom) #5 - Myocardium circularity
    XTrain.loc[k,'circum_ED_mean'], XTrain.loc[k,'circum_ED_max'], XTrain.loc[k,'circum_ED_min'] = circumference_myo((data == 2),zoom) #6 - Myocardium circumference
    XTrain.loc[k,'r_rv_lv_ED'] = XTrain.loc[k,'rv_ED'] / XTrain.loc[k,'lv_ED'] #7 - Ratio between RV and LV
    XTrain.loc[k, 'r_myo_lv_ED'] = XTrain.loc[k,'myo_ED'] / XTrain.loc[k,'lv_ED'] #8 - Ratio between Myocardium and LV

    # ES
    data = XTrainImg[3,k].get_fdata()
    zoom = np.array(XTrainImg[3,k].header.get_zooms())
    XTrain.loc[k,'rv_ES'] = compute_vol((data == 1), zoom) #1 - Left ventricle cavity
    XTrain.loc[k,'myo_ES'] = compute_vol((data == 2), zoom) #2 - Myocardium
    XTrain.loc[k,'lv_ES'] = compute_vol((data == 3), zoom) #3 - Right ventricle cavity
    XTrain.loc[k,'thick_ES_mean'], XTrain.loc[k,'thick_ES_std'], XTrain.loc[k,'thick_ES_max'], XTrain.loc[k,'thick_ES_min'] = thickness_myo((data == 2), zoom) #4 - Myocardium thickness
    XTrain.loc[k,'circul_ES'] = circularity((data == 2), zoom) #5 - Myocardium circularity
    XTrain.loc[k,'circum_ES_mean'], XTrain.loc[k,'circum_ES_max'], XTrain.loc[k,'circum_ES_min'] = circumference_myo((data == 2), zoom) #6 - Myocardium circumference
    XTrain.loc[k,'r_rv_lv_ES'] = XTrain.loc[k,'rv_ES'] / XTrain.loc[k,'lv_ES'] #7 - Ratio between RV and LV
    XTrain.loc[k, 'r_myo_lv_ES'] = XTrain.loc[k,'myo_ES'] / XTrain.loc[k,'lv_ES'] #8 - Ratio between Myocardium and LV

    # Dynamic features
    #add Ejection Fraction
    XTrain.loc[k,'lv_EF'] = (XTrain.loc[k,'lv_ED'] - XTrain.loc[k,'lv_ES']) / XTrain.loc[k,'lv_ED']
    XTrain.loc[k,'rv_EF'] = (XTrain.loc[k,'rv_ED'] - XTrain.loc[k,'rv_ES']) / XTrain.loc[k,'rv_ED']

    #median, standard deviation, kurtosis, skewness of the volume
    myo_ED = (XTrainImg[1,k].get_fdata() == 2).astype(np.uint8)
    myo_ES = (XTrainImg[3,k].get_fdata() == 2).astype(np.uint8)
    elem_vol = zoom[0] * zoom[1] * zoom[2]
    XTrain.loc[k,'myo_vol_median'] = np.median(myo_ED - myo_ES) * elem_vol
    XTrain.loc[k,'myo_vol_std'] = np.std(myo_ED - myo_ES) * elem_vol
    XTrain.loc[k,'myo_vol_kurtosis'] = kurtosis(myo_ED - myo_ES, axis=None)
    XTrain.loc[k,'myo_vol_skewness'] = skew(myo_ED - myo_ES, axis=None)

labels = XTrain.columns.values.tolist()

#adding data for test
for k in range (0,XTest.shape[0]):
    
    k_str = '{:03d}'.format(k + XTrain.shape[0] + 1)
    ED = '.\\Test\\' + k_str + '\\' + k_str + '_ED.nii'
    ED_seg = '.\\Test\\' + k_str + '\\' + k_str + '_ED_seg.nii'
    ES = '.\\Test\\' + k_str + '\\' + k_str + '_ES.nii'
    ES_seg = '.\\Test\\' + k_str + '\\' + k_str + '_ES_seg.nii'
    
    XTestImg[0,k] = nib.load(ED)
    XTestImg[1,k] = nib.load(ED_seg)
    XTestImg[2,k] = nib.load(ES)
    XTestImg[3,k] = nib.load(ES_seg)

    # ED
    data = XTestImg[1,k].get_fdata()
    zoom = np.array(XTestImg[1,k].header.get_zooms())
    #add seg
    data += 3*mask_contour(data == 2)
    XTest.loc[k,'rv_ED'] = compute_vol((data == 1),zoom) #1 - Left ventricle cavity
    XTest.loc[k,'myo_ED'] = compute_vol((data == 2), zoom) #2 - Myocardium
    XTest.loc[k,'lv_ED'] = compute_vol((data == 3), zoom) #3 - Right ventricle cavity
    XTest.loc[k,'thick_ED_mean'], XTest.loc[k,'thick_ED_std'], XTest.loc[k,'thick_ED_max'], XTest.loc[k,'thick_ED_min'] = thickness_myo((data == 2), zoom) #4 - Myocardium thickness Mean
    XTest.loc[k,'circul_ED'] = circularity((data == 2), zoom) #5 - Myocardium circularity
    XTest.loc[k,'circum_ED_mean'], XTest.loc[k,'circum_ED_max'], XTest.loc[k,'circum_ED_min'] = circumference_myo((data == 2), zoom) #6 - Myocardium circumference
    XTest.loc[k,'r_rv_lv_ED'] = XTest.loc[k,'rv_ED'] / XTest.loc[k,'lv_ED'] #7 - Ratio between RV and LV
    XTest.loc[k, 'r_myo_lv_ED'] = XTest.loc[k,'myo_ED'] / XTest.loc[k,'lv_ED'] #8 - Ratio between Myocardium and LV

    # ES
    data = XTestImg[3,k].get_fdata()
    zoom = np.array(XTestImg[3,k].header.get_zooms())
    #add seg
    data += 3*mask_contour(data == 2)
    XTest.loc[k,'rv_ES'] = compute_vol((data == 1), zoom) #1 - Left ventricle cavity
    XTest.loc[k,'myo_ES'] = compute_vol((data == 2), zoom) #2 - Myocardium
    XTest.loc[k,'lv_ES'] = compute_vol((data == 3), zoom) #3 - Right ventricle cavity
    XTest.loc[k,'thick_ES_mean'], XTest.loc[k,'thick_ES_std'], XTest.loc[k,'thick_ES_max'], XTest.loc[k,'thick_ES_min'] = thickness_myo((data == 2), zoom) #4 - Myocardium thickness
    XTest.loc[k,'circul_ES'] = circularity((data == 2), zoom) #5 - Myocardium circularity
    XTest.loc[k,'circum_ES_mean'], XTest.loc[k,'circum_ES_max'], XTest.loc[k,'circum_ES_min'] = circumference_myo((data == 2), zoom) #6 - Myocardium circumference
    XTest.loc[k,'r_rv_lv_ES'] = XTest.loc[k,'rv_ES'] / XTest.loc[k,'lv_ES'] #7 - Ratio between RV and LV
    XTest.loc[k, 'r_myo_lv_ES'] = XTest.loc[k,'myo_ES'] / XTest.loc[k,'lv_ES'] #8 - Ratio between Myocardium and LV
    
    # Ejection Fraction
    XTest.loc[k,'lv_EF'] = (XTest.loc[k,'lv_ED'] - XTest.loc[k,'lv_ES']) / XTest.loc[k,'lv_ED']
    XTest.loc[k,'rv_EF'] = (XTest.loc[k,'rv_ED'] - XTest.loc[k,'rv_ES']) / XTest.loc[k,'rv_ED']

    #median, standard deviation, kurtosis, skewness of the volume
    myo_ED = (XTestImg[1,k].get_fdata() == 2).astype(np.uint8)
    myo_ES = (XTestImg[3,k].get_fdata() == 2).astype(np.uint8)
    elem_vol = zoom[0] * zoom[1] * zoom[2]
    XTest.loc[k,'myo_vol_median'] = np.median(myo_ED - myo_ES) * elem_vol
    XTest.loc[k,'myo_vol_std'] = np.std(myo_ED - myo_ES) * elem_vol
    XTest.loc[k,'myo_vol_kurtosis'] = kurtosis(myo_ED - myo_ES, axis=None)
    XTest.loc[k,'myo_vol_skewness'] = skew(myo_ED - myo_ES, axis=None)

## 4. Algorithms used for prediction
### a. Random Forest Model

In [9]:
L = ['rv_ED', 
     'myo_ED', 
     'lv_ED', 
     'thick_ED_mean', 
     'thick_ED_max',
     'r_rv_lv_ED',
     'r_myo_lv_ED',
     'rv_ES',
     'lv_ES',
     'thick_ES_mean',
     'thick_ES_max',
     'r_rv_lv_ES',
     'r_myo_lv_ES',
     'rv_EF']

x_train = XTrain
y_train = yTrain
x_test = XTest

Forest = RandomForestClassifier(n_estimators=1000, random_state = 0)
Forest.fit(x_train[L], y_train)
scores = cross_val_score(Forest, x_train[L], y_train, cv=7)
y_pred = Forest.predict(x_test[L])

### b. Gradient Boosting Classifier to classified MINF and DCM

In [10]:
L = ['rv_ED', 
     'myo_ED', 
     'lv_ED', 
     'r_rv_lv_ED',
     'r_myo_lv_ED',
     'rv_ES',
     'myo_ES',
     'lv_ES',
     'thick_ES_mean',
     'r_rv_lv_ES',
     'r_myo_lv_ES',
     'lv_EF',
     'rv_EF']

# Select samples classified as MINF and DCM
x_train_2 = x_train[(y_train == 1) | (y_train == 2)]
y_train_2 = y_train[(y_train == 1) | (y_train == 2)]
x_test_2 = x_test[(y_pred == 1) | (y_pred == 2)]

# Scale the data
scaler = StandardScaler()
scaler.fit(x_train_2[L])
x_train_2.loc[:, L] = scaler.transform(x_train_2[L])
x_test_2.loc[:, L] = scaler.transform(x_test_2[L])

# Gradient Boosting Classifier
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1, random_state=34)
model.fit(x_train_2[L], y_train_2)
scores = cross_val_score(model, x_train_2[L], y_train_2, cv=7)
y_pred_2 = model.predict(x_test_2[L])

### c. Merge the classification of Gradient Boosting Classifier and Random Forest Classifier, and export the result

In [11]:
# merge the classifiers
y_pred[(y_pred == 1) | (y_pred == 2)] = y_pred_2

# Export the predictions to a csv file for submission
y_pred = y_pred.astype(int)
df = pd.DataFrame({'Id': Id, 'Category': y_pred})
df.to_csv('submission.csv', index=False)

## 4. Other algorithms tested

### a. Multi-Level perceptron (MLP) (not used in the Final Model)

The first is a MLP inspired by one of the articles. It doesn't give good results, but I keep it as a trace of my research.

In [12]:
# Set up MLP architecture
mlp_architecture = (32, 32, 32, 32)
mlp_batch_size = 40
mlp_epochs = 4000
mlp_learning_rate = 5e-4

L = ['rv_ED', 
     'myo_ED', 
     'lv_ED', 
     'thick_ED_mean', 
     'thick_ED_max',
     'r_rv_lv_ED',
     'r_myo_lv_ED',
     'rv_ES',
     'lv_ES',
     'thick_ES_mean',
     'thick_ES_max',
     'r_rv_lv_ES',
     'r_myo_lv_ES',
     'rv_EF']

# Set up training and testing data with a 75/25 split
x_train, x_test, y_train, y_test = train_test_split(XTrain[L], yTrain, test_size=0.2, random_state=42)

#convert to numpy array
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

# Initialize MLP and Random Forest models
mlps = []
for i in range(50):
    mlp = MLPClassifier(hidden_layer_sizes=mlp_architecture,
                        activation='relu',
                        alpha=0.0001,
                        batch_size=mlp_batch_size,
                        learning_rate_init=mlp_learning_rate,
                        max_iter=mlp_epochs,
                        shuffle=True,
                        random_state=i)
    mlps.append(mlp)

# Train each MLP on a random subset of the training data
for i in range(50):
    # Randomly select 2/3 of the training data
    subset = np.random.choice(x_train.shape[0], size=int(0.66 * x_train.shape[0]), replace=False)
    x_subset = x_train[subset]
    y_subset = y_train[subset]
    # Only present a random subset of 2/3 of the features to each MLP
    features_subset = np.random.choice(x_train.shape[1], size=int(0.66 * x_train.shape[1]), replace=False)
    x_subset = x_subset[:, features_subset]
    # Scale the data and fit the MLP
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x_subset)
    mlps[i].fit(x_scaled, y_subset)

# Evaluate MLP ensemble on test data
mlp_predictions = []
for i in range(50):
    # Scale the test data
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x_test)
    # Only use a random subset of 2/3 of the features
    x_scaled = x_scaled[:, features_subset]
    # Make predictions
    mlp_predictions.append(mlps[i].predict(x_scaled))

# Ensemble predictions
mlp_predictions = np.array(mlp_predictions)
mlp_predictions = np.swapaxes(mlp_predictions, 0, 1)
ensemble_predictions = []
for i in range(mlp_predictions.shape[0]):
    ensemble_predictions.append(np.argmax(np.bincount(mlp_predictions[i])))
ensemble_predictions = np.array(ensemble_predictions)